### Habilitación de las conexiones Cross-VPC transitivas mediante gateway de tránsito

#### Problema
Necesita implementar el enrutamiento transitivo a través de todas sus VPCs y compartir la salida de Internet desde una VPC de servicios compartidos a sus otras VPCs para reducir el número de gateways NAT que tiene que implementar. 

#### Solución
Implemente una gateway de tránsito de AWS (TGW) y configure los anexos de la VPC de la gateway de tránsito para todas sus VPC. Actualice sus tablas de rutas de VPC de cada VPC para enviar todo el tráfico no local a la gateway de tránsito y habilite el uso compartido de la gateway de NAT en su VPC de servicios compartidos para todas sus VPCs de radio.
<br>
<br>
<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0214.png" width="600">

**NOTA:** La cuota inicial por defecto de VPCs por Región por cuenta es de cinco. Esta solución desplegará tres VPCs. Si ya tiene más de dos VPC, puede decidir entre cuatro opciones: desplegar en una Región diferente, eliminar cualquier VPC existente que ya no sea necesaria, utilizar una cuenta de prueba o solicitar un aumento de cuota.

In [1]:
import boto3
import json
import time

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)

In [2]:
# create 3 VPC
vpc1 = ec2.create_vpc(CidrBlock='10.10.0.0/16')
vpc1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1"}])
vpc2 = ec2.create_vpc(CidrBlock='10.11.0.0/16')
vpc2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2"}])
vpc3 = ec2.create_vpc(CidrBlock='10.12.0.0/16')
vpc3.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-3"}])
vpc3.wait_until_available()

In [3]:
# Crear 4 subredes para VPC1
vpc1_subnet_private1 = ec2.create_subnet(
    CidrBlock='10.10.0.0/24', 
    VpcId=vpc1.id,
    AvailabilityZone=region_aws+'a'
)
vpc1_subnet_private2 = ec2.create_subnet(
    CidrBlock='10.10.1.0/24',
    VpcId=vpc1.id,
    AvailabilityZone=region_aws+'b'
)
vpc1_subnet_private_att_1 = ec2.create_subnet(
    CidrBlock='10.10.2.0/24', 
    VpcId=vpc1.id,
    AvailabilityZone=region_aws+'a'
)
vpc1_subnet_private_att_2 = ec2.create_subnet(
    CidrBlock='10.10.3.0/24',
    VpcId=vpc1.id,
    AvailabilityZone=region_aws+'b'
)

In [4]:
# Create tags
vpc1_subnet_private1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1-Private-1"}])
vpc1_subnet_private2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1-Private-2"}])
vpc1_subnet_private_att_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1-Private-Attach-1"}])
vpc1_subnet_private_att_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1-Private-Attach-2"}])

[ec2.Tag(resource_id='subnet-0996ccc4df7d05795', key='Name', value='AWSCookBookVPC-1-Private-Attach-2')]

In [5]:
# Cree 6 subredes para VPC2:
vpc2_subnet_public1 = ec2.create_subnet(
    CidrBlock='10.11.0.0/24', 
    VpcId=vpc2.id,
    AvailabilityZone=region_aws+'a'
)
vpc2_subnet_public2 = ec2.create_subnet(
    CidrBlock='10.11.1.0/24',
    VpcId=vpc2.id,
    AvailabilityZone=region_aws+'b'
)
vpc2_subnet_private1 = ec2.create_subnet(
    CidrBlock='10.11.2.0/24', 
    VpcId=vpc2.id,
    AvailabilityZone=region_aws+'a'
)
vpc2_subnet_private2 = ec2.create_subnet(
    CidrBlock='10.11.3.0/24',
    VpcId=vpc2.id,
    AvailabilityZone=region_aws+'b'
)
vpc2_subnet_private_att_1 = ec2.create_subnet(
    CidrBlock='10.11.4.0/24',
    VpcId=vpc2.id,
    AvailabilityZone=region_aws+'a'
)
vpc2_subnet_private_att_2 = ec2.create_subnet(
    CidrBlock='10.11.5.0/24',
    VpcId=vpc2.id,
    AvailabilityZone=region_aws+'b'
)

In [6]:
# Create tags
vpc2_subnet_public1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Public-1"}])
vpc2_subnet_public2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Public-2"}])
vpc2_subnet_private1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Private-1"}])
vpc2_subnet_private2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Private-2"}])
vpc2_subnet_private_att_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Private-Attach-1"}])
vpc2_subnet_private_att_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Private-Attach-2"}])

[ec2.Tag(resource_id='subnet-07ca98fa90ea72263', key='Name', value='AWSCookBookVPC-2-Private-Attach-2')]

In [7]:
# Cree 4 subredes para VPC3:
vpc3_subnet_private1 = ec2.create_subnet(
    CidrBlock='10.12.0.0/24', 
    VpcId=vpc3.id,
    AvailabilityZone=region_aws+'a'
)
vpc3_subnet_private2 = ec2.create_subnet(
    CidrBlock='10.12.1.0/24',
    VpcId=vpc3.id,
    AvailabilityZone=region_aws+'b'
)
vpc3_subnet_private_att_1 = ec2.create_subnet(
    CidrBlock='10.12.2.0/24', 
    VpcId=vpc3.id,
    AvailabilityZone=region_aws+'a'
)
vpc3_subnet_private_att_2 = ec2.create_subnet(
    CidrBlock='10.12.3.0/24',
    VpcId=vpc3.id,
    AvailabilityZone=region_aws+'b'
)

In [8]:
# Create tags
vpc3_subnet_private1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-3-Private-1"}])
vpc3_subnet_private2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-3-Private-2"}])
vpc3_subnet_private_att_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-3-Private-Attach-1"}])
vpc3_subnet_private_att_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-3-Private-Attach-2"}])

[ec2.Tag(resource_id='subnet-0c12ba489e8cc1311', key='Name', value='AWSCookBookVPC-3-Private-Attach-2')]

In [9]:
# Crear routables para VPC1 - private
vpc1_route_table_private_1 = ec2.create_route_table(VpcId=vpc1.id)
vpc1_route_table_private_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1-Private-Route-Table-a"}])
vpc1_route_table_private_2 = ec2.create_route_table(VpcId=vpc1.id)
vpc1_route_table_private_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1-Private-Route-Table-b"}])
# Crear routables para VPC1 - attach
vpc1_route_table_private_att_1 = ec2.create_route_table(VpcId=vpc1.id)
vpc1_route_table_private_att_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1-Private-Attach-Route-Table-a"}])
vpc1_route_table_private_att_2 = ec2.create_route_table(VpcId=vpc1.id)
vpc1_route_table_private_att_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-1-Private-Attach-Route-Table-b"}])

# Crear routables para VPC2 - public
vpc2_route_table_public = ec2.create_route_table(VpcId=vpc2.id)
vpc2_route_table_public.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-Public-Route-Table"}])
# Crear routables para VPC2 - private
vpc2_route_table_private_1 = ec2.create_route_table(VpcId=vpc2.id)
vpc2_route_table_private_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Private-Route-Table-a"}])
vpc2_route_table_private_2 = ec2.create_route_table(VpcId=vpc2.id)
vpc2_route_table_private_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Private-Route-Table-b"}])
# Crear routables para VPC2 - attach
vpc2_route_table_private_att_1 = ec2.create_route_table(VpcId=vpc2.id)
vpc2_route_table_private_att_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Private-Attach-Route-Table-a"}])
vpc2_route_table_private_att_2 = ec2.create_route_table(VpcId=vpc2.id)
vpc2_route_table_private_att_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-2-Private-Attach-Route-Table-b"}])

# Crear routables para VPC3 - private
vpc3_route_table_private_1 = ec2.create_route_table(VpcId=vpc3.id)
vpc3_route_table_private_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-3-Private-Route-Table-a"}])
vpc3_route_table_private_2 = ec2.create_route_table(VpcId=vpc3.id)
vpc3_route_table_private_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-3-Private-Route-Table-b"}])
# Crear routables para VPC3 - attach
vpc3_route_table_private_att_1 = ec2.create_route_table(VpcId=vpc3.id)
vpc3_route_table_private_att_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-3-Private-Attach-Route-Table-a"}])
vpc3_route_table_private_att_2 = ec2.create_route_table(VpcId=vpc3.id)
vpc3_route_table_private_att_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC-3-Private-Attach-Route-Table-b"}])


[ec2.Tag(resource_id='rtb-0bfaba1339cd8b54e', key='Name', value='AWSCookBookVPC-3-Private-Route-Table-b')]

In [10]:
# Asoociar la tabla de rutas a la subred privada de VPC1
vpc1_route_table_private_1.associate_with_subnet(SubnetId=vpc1_subnet_private1.id)
vpc1_route_table_private_2.associate_with_subnet(SubnetId=vpc1_subnet_private2.id)
vpc1_route_table_private_att_1.associate_with_subnet(SubnetId=vpc1_subnet_private_att_1.id)
vpc1_route_table_private_att_2.associate_with_subnet(SubnetId=vpc1_subnet_private_att_2.id)


# Asoociar la tabla de rutas a la subred privada de VPC2
vpc2_route_table_private_1.associate_with_subnet(SubnetId=vpc2_subnet_private1.id)
vpc2_route_table_private_2.associate_with_subnet(SubnetId=vpc2_subnet_private2.id)
vpc2_route_table_private_att_1.associate_with_subnet(SubnetId=vpc2_subnet_private_att_1.id)
vpc2_route_table_private_att_2.associate_with_subnet(SubnetId=vpc2_subnet_private_att_2.id)


# Asoociar la tabla de rutas a la subred privada de VPC3
vpc3_route_table_private_1.associate_with_subnet(SubnetId=vpc3_subnet_private1.id)
vpc3_route_table_private_2.associate_with_subnet(SubnetId=vpc3_subnet_private2.id)
vpc3_route_table_private_att_1.associate_with_subnet(SubnetId=vpc3_subnet_private_att_1.id)
vpc3_route_table_private_att_2.associate_with_subnet(SubnetId=vpc3_subnet_private_att_2.id)


# Asoociar la tabla de rutas a la subred publica de VPC2
vpc2_route_table_public.associate_with_subnet(SubnetId=vpc2_subnet_public1.id)
vpc2_route_table_public.associate_with_subnet(SubnetId=vpc2_subnet_public2.id)

ec2.RouteTableAssociation(id='rtbassoc-0cf654ab950c0b76e')

In [11]:
# Create 2 elastic IP
eip1 = ec2_client.allocate_address(Domain='vpc')
eip2 = ec2_client.allocate_address(Domain='vpc')

In [12]:
# Create 2 NAT Gateways
nat1 = ec2_client.create_nat_gateway(
    AllocationId=eip1['AllocationId'],
    SubnetId=vpc2_subnet_public1.id
)
nat2 = ec2_client.create_nat_gateway(
    AllocationId=eip2['AllocationId'],
    SubnetId=vpc2_subnet_public2.id
)

In [13]:
# create 1 internet gateways
igw = ec2.create_internet_gateway()
igw.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookIGW"}])

[ec2.Tag(resource_id='igw-0788087edb99e361a', key='Name', value='AWSCookBookIGW')]

In [ ]:
# attach internet gateway to VPC N° 2
response = vpc2.attach_internet_gateway(InternetGatewayId=igw.id)

# Cree una ruta de Internet para la tabla de enrutamiento pública:
vpc2_route_table_public.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=igw.id
)

#### Crear un gateway de tránsito

In [15]:
tgw = ec2_client.create_transit_gateway(
    Description='AWSCookBookTGW',
    Options={
        'AmazonSideAsn': 65010,
        'AutoAcceptSharedAttachments': 'enable',
        'DefaultRouteTableAssociation': 'enable',
        'DefaultRouteTablePropagation': 'enable',
        'DnsSupport': 'enable',
        'VpnEcmpSupport': 'enable'
    }
)

In [16]:
# Tag the transit gateway
response = ec2_client.create_tags(
    Resources=[tgw['TransitGateway']['TransitGatewayId']],
    Tags=[{'Key': 'Name', 'Value': 'AWSCookBookTGW'}]
)

Espere hasta que el estado del gateway de tránsito esté disponible. Esto puede tardar varios minutos:

In [19]:
state = "pending"
print(tgw['TransitGateway']['State'] + "...")
while state == "pending":
    # Describe the transit gateway
    tgw_state = ec2_client.describe_transit_gateways(
        TransitGatewayIds=[
            tgw['TransitGateway']['TransitGatewayId'],
        ]
    )
    state = tgw_state['TransitGateways'][0]['State']
print(tgw_state['TransitGateways'][0]['State'])

pending...
available


In [20]:
# Create a transit gateway attachment for VPC1:
tgw_attachment_vpc1 = ec2_client.create_transit_gateway_vpc_attachment(
    TransitGatewayId=tgw['TransitGateway']['TransitGatewayId'],
    VpcId=vpc1.id,
    SubnetIds=[
        vpc1_subnet_private_att_1.id,
        vpc1_subnet_private_att_2.id,
    ],
)

In [21]:
# Create a transit gateway attachment for VPC2:
tgw_attachment_vpc2 = ec2_client.create_transit_gateway_vpc_attachment(
    TransitGatewayId=tgw['TransitGateway']['TransitGatewayId'],
    VpcId=vpc2.id,
    SubnetIds=[
        vpc2_subnet_private_att_1.id,
        vpc2_subnet_private_att_2.id,
    ],
)

In [22]:
# Create a transit gateway attachment for VPC3:
tgw_attachment_vpc3 = ec2_client.create_transit_gateway_vpc_attachment(
    TransitGatewayId=tgw['TransitGateway']['TransitGatewayId'],
    VpcId=vpc3.id,
    SubnetIds=[
        vpc3_subnet_private_att_1.id,
        vpc3_subnet_private_att_2.id,
    ],
)

In [23]:
# Create Tags for the transit gateway attachments
response = ec2_client.create_tags(
    Resources=[
        tgw_attachment_vpc1['TransitGatewayVpcAttachment']['TransitGatewayAttachmentId']
    ],
    Tags=[
        {'Key': 'Name', 'Value': 'AWSCookBookTGW-Attachment-VPC1'}
    ]
)

response = ec2_client.create_tags(
    Resources=[
        tgw_attachment_vpc2['TransitGatewayVpcAttachment']['TransitGatewayAttachmentId']
    ],
    Tags=[
        {'Key': 'Name', 'Value': 'AWSCookBookTGW-Attachment-VPC2'}
    ]
)

response = ec2_client.create_tags(
    Resources=[
        tgw_attachment_vpc3['TransitGatewayVpcAttachment']['TransitGatewayAttachmentId']
    ],
    Tags=[
        {'Key': 'Name', 'Value': 'AWSCookBookTGW-Attachment-VPC3'}
    ]
)

Añade rutas para todas las subredes privadas en las VPCs 1 y 3 para apuntar al TGW para destinos de 0.0.0.0/0. Esto permite la salida consolidada de Internet a través de la puerta de enlace NAT en la VPC2 y el enrutamiento transitivo a otras VPCs:

In [27]:
vpc1_route_table_private_1.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=tgw['TransitGateway']['TransitGatewayId']
)
vpc1_route_table_private_2.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=tgw['TransitGateway']['TransitGatewayId']
)

ec2.Route(route_table_id='rtb-051746db12798b38f', destination_cidr_block='0.0.0.0/0')

In [28]:
vpc3_route_table_private_1.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=tgw['TransitGateway']['TransitGatewayId']
)
vpc3_route_table_private_2.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=tgw['TransitGateway']['TransitGatewayId']
)

ec2.Route(route_table_id='rtb-0bfaba1339cd8b54e', destination_cidr_block='0.0.0.0/0')

Ahora añade una ruta a tu superred 10.10.0.0/24 en las tablas de rutas asociadas a las subredes privadas de la VPC2, apuntando su destino a la gateway de tránsito. Esto es más específico que el destino 0.0.0.0/0 que ya está presente y, por lo tanto, tiene mayor prioridad en las decisiones de enrutamiento. Esto dirige el tráfico con destino a las VPC 1, 2 y 3 al TGW:

In [29]:
vpc2_route_table_private_1.create_route(
    DestinationCidrBlock='10.10.0.0/24',
    TransitGatewayId=tgw['TransitGateway']['TransitGatewayId']
)
vpc2_route_table_private_2.create_route(
    DestinationCidrBlock='10.10.0.0/24',
    TransitGatewayId=tgw['TransitGateway']['TransitGatewayId']
)

ec2.Route(route_table_id='rtb-047d045989df5034b', destination_cidr_block='10.10.0.0/24')

Consulta las gateways NAT en uso; las necesitaremos para añadir rutas a ellas para el tráfico de Internet:

In [30]:
nat1_id = nat1['NatGateway']['NatGatewayId']
nat2_id = nat2['NatGateway']['NatGatewayId']

In [31]:
# Add a route for the attachment subnet in VPC2 to direct internet traffic to the NAT gateway:
vpc2_route_table_private_att_1.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    NatGatewayId=nat1_id
)
vpc2_route_table_private_att_2.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    NatGatewayId=nat2_id
)

ec2.Route(route_table_id='rtb-047d045989df5034b', destination_cidr_block='0.0.0.0/0')

In [33]:
# Add a static route to the route tables associated with the public subnet in VPC2. This enables communication back to the TGW to allow sharing the NAT gateway with all attached VPCs:
vpc2_route_table_public.create_route(
    DestinationCidrBlock='10.10.0.0/24',
    TransitGatewayId=tgw['TransitGateway']['TransitGatewayId']
)

ec2.Route(route_table_id='rtb-0a848a3cf99ff10ce', destination_cidr_block='10.10.0.0/24')

In [34]:
# Add a static route for the private subnets in VPC2 to allow communication back to the TGW attachments from VPC2 private subnets:
vpc2_route_table_private_1.create_route(
    DestinationCidrBlock='10.10.0.0/24',
    TransitGatewayId=tgw['TransitGateway']['TransitGatewayId']
)
vpc2_route_table_private_2.create_route(
    DestinationCidrBlock='10.10.0.0/24',
    TransitGatewayId=tgw['TransitGateway']['TransitGatewayId']
)

ec2.Route(route_table_id='rtb-047d045989df5034b', destination_cidr_block='10.10.0.0/24')

In [44]:
# Get the transit route table ID:
response = ec2_client.describe_transit_gateways(
    TransitGatewayIds=[
        tgw['TransitGateway']['TransitGatewayId']
    ]
)
tgw_route_table_id = response['TransitGateways'][0]['Options']['AssociationDefaultRouteTableId']

In [45]:
# Add a static route in the transit gateway route table for VPC2 (with the NAT gateways) to send all internet traffic over this path:
response = ec2_client.create_transit_gateway_route(
    TransitGatewayRouteTableId=tgw_route_table_id,
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayAttachmentId=tgw_attachment_vpc2['TransitGatewayVpcAttachment']['TransitGatewayAttachmentId']
)

#### Controles de validación

Asegúrese de que su instancia EC2 1 se ha registrado en SSM. Utilize el siguiente comando en la terminal:

`python create_ec2_ssm.py --vpc <vpc-id> --subnet <subnet-id>`

Y en la instancia:

`ping -c 4 aws.amazon.com`

#### Reto 1
Puede limitar qué VPC puede acceder a Internet a través de la gateway NAT en VPC2 modificando las tablas de rutas. Intente añadir una ruta más específica de 10.10.0.0/24 en lugar del destino 0.0.0/0 para la VPC3 para ver cómo puede personalizar el reparto de la salida a Internet.

#### Reto 2
Es posible que no quiera permitir que VPC1 y VPC3 puedan comunicarse entre sí. Intente añadir una nueva tabla de rutas de gateway de tránsito, actualizando los anexos para conseguirlo.

#### Reto 3
En la solución, usted desplegó tres VPCs cada una de tamaño /26 de subred dentro de la superred 10.10.0.0/24. Hay espacio para una subred /26 adicional. Intente añadir una VPC adicional con un CIDR /26 con subredes y tablas de rutas; a continuación, adjúntela a la gateway de tránsito.

### Discusión

Los gateway de tránsito le permiten implementar rápidamente una topología de red multi-VPC hub-and-spoke para su red en AWS. Es posible que haya tenido que utilizar muchas conexiones de peering para lograr resultados similares o que haya utilizado software de terceros en instancias en una arquitectura VPC de tránsito. La gateway de tránsito también admite el peering entre regiones de las gateways de tránsito y el uso compartido entre cuentas a través de Resource Access Manager (RAM).

Cuando adjuntó sus VPC a la gateway de tránsito, utilizó subredes en cada AZ para la resiliencia. También utilizó subredes de "adjunto" dedicadas para los adjuntos de la VPC. Puede adjuntar el gateway de tránsito a cualquier subred dentro de su VPC. El uso de una subred dedicada para estos anexos le da flexibilidad para definir de forma granular las subredes que elija para enrutar a la TGW. Es decir, si usted adjunta la subred privada, ésta siempre tendrá una ruta hacia el TGW; esto podría no ser la intención en base a su caso de uso. En tu caso, has configurado las rutas de tus subredes privadas para que envíen todo el tráfico a la gateway de tránsito, lo que ha permitido compartir la gateway de NAT y la de Internet; esto supone un ahorro de costes respecto a tener que desplegar varias gateways de NAT (por ejemplo, una para cada VPC).

Puede conectar su red local o cualquier red virtual directamente a un gateway de tránsito, ya que éste actúa como un centro para todo su tráfico de red de AWS. Puede conectar VPNs IPsec, Direct Connect (DX) y dispositivos de red de terceros a la gateway de tránsito para extender su red AWS a redes que no sean AWS. Esto también le permite consolidar conexiones VPN y/o conexiones Direct Connect conectando una directamente a la gateway de tránsito para acceder a todas sus VPC en una región. TGW admite el protocolo de gateway de frontera (BGP) en estos tipos de extensiones de red para las actualizaciones de rutas dinámicas en ambas direcciones.

#### Desafío
Cree una cuarta VPC y adjunte su TGW a las subredes en ella. Permita que utilice el gateway NAT existente para llegar a Internet.